In [2]:
import pandas as pd
import re
from textblob import TextBlob
import spacy as spacy
import numpy as np
nlp = spacy.load("pl_core_news_lg")
from spacy.lang.pl import Polish
parser = Polish()
from spacy.lang.pl.stop_words import STOP_WORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit

In [3]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [4]:
stopwords = list(STOP_WORDS)

# Cleaning and importing data


In [5]:
import string
punctuations=string.punctuation

In [6]:
def tokenizer(sentence):
    mytokens=nlp(sentence)
    mytokens=[word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [7]:
c=tokenizer('W celu pokazu prezentuję następujące zdanie')
c

['cel', 'pokaz', 'prezentować', 'następujący', 'zdanie']

In [9]:
vectorizer=CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

In [10]:
df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')
df

<ipython-input-10-4df8a0abd9e6>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')


,Text,label,label;;;;;;
0,"Taksowak przyjezdza na czas , wydaje sie , ze ...",label,z_zero;;;;;;
1,Do tej pani trafili śmy z naszą 2 - miesięczna...,label,z_zero;;;;;;
2,"Wyróżnikiem , a także ciekawym elementem całeg...",label,z_zero;;;;;;
3,"W zależności od pory dnia , w naszych organizm...",label,z_zero;;;;;;
4,"Polecam wszystkim , zwłaszcza tym , dla któryc...",label,z_plus_m;;;;;;
...,...,...,...
45959,Na plus zaliczył by m też wbudowane gniazda US...,label,z_plus_m;;;;;;
45960,Bardzo blisko ( 3 - 5 min . drogi ) do centrum...,label,z_plus_m;;;;;;
45961,"Fachowo , prawie bezboleśnie i skutecznie , bo...",label,z_plus_m;;;;;;
45962,Podsumowując : 1 . umówił em się na prywatną w...,label,z_minus_m;;;;;;


In [11]:
def getRidOff(text):
    text=re.sub(r';','',text)
    text=re.sub(r'"','',text)
    return text
df['label;;;;;;']=df['label;;;;;;'].apply(getRidOff)

In [12]:
def cleanText(text):
    text=re.sub(r':','',text)
    text=re.sub(r';','',text)
    text=re.sub(r'[0-9]+','',text)
    text=re.sub(r'  ',' ',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'-','',text)
    text=re.sub(r',','',text)
    text=re.sub(r'"','',text)
    return text
df['Text']=df['Text'].apply(cleanText)

In [13]:
df.drop('label',axis=1,inplace=True)

In [14]:
df=df.rename(columns={'label;;;;;;':'label'})
df.to_csv('daneImport.csv')

In [15]:
df =df[df.label != 'z_zero']
df =df[df.label != 'z_amb']
dfbb

,Text,label
4,Polecam wszystkim zwłaszcza tym dla których ...,z_plus_m
6,Wszystko stare i okropne .,z_minus_m
7,Arogancki właściciel ustawiający gości .,z_minus_m
10,Pokój relaksu z Jacuzzi różnorodne usługi ma...,z_plus_m
11,Hotel na bardzo dobrym poziomie zarówno jeśli ...,z_plus_m
...,...,...
45958,W ostatni dzień kucharz podał nam surowego klo...,z_minus_m
45959,Na plus zaliczył by m też wbudowane gniazda US...,z_plus_m
45960,Bardzo blisko ( min . drogi ) do centrum i ma...,z_plus_m
45961,Fachowo prawie bezboleśnie i skutecznie bo e...,z_plus_m


# Splitting data and training model


In [28]:
X=df['Text']

In [29]:
y=df['label']


In [30]:
y

4         z_plus_m
6        z_minus_m
7        z_minus_m
10        z_plus_m
11        z_plus_m
           ...    
45958    z_minus_m
45959     z_plus_m
45960     z_plus_m
45961     z_plus_m
45962    z_minus_m
Name: label, Length: 29200, dtype: object

In [31]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
pipe = Pipeline([('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [33]:
X_train

28460            Pokoje bardzo czyste  ładnie urządzone . 
12234    Bardzo polecam wszystkim kobietom kontakt z ty...
25443    Dodam  że mama była ciężko chora na nowotwór i...
1068     Zarowno lek jak i srodki do pielegnacji skory ...
26237      Obsługa miła  wszyscy uśmiechnięci i pomocni . 
                               ...                        
33928    Na stołówce notorycznie czegoś brakowało taler...
8442     Moim zdaniem nadaje się do interwencji Izby Le...
1389     Wystraszym mnie nieziemsko  zakazal kontynuacj...
24885                 Ogólne bardzo słabo i nie polecam . 
37200    Zadał tylko ogólnikowe pytania typu gdzie mies...
Name: Text, Length: 23360, dtype: object

In [34]:
Y_train

28460     z_plus_m
12234     z_plus_m
25443    z_minus_m
1068     z_minus_m
26237     z_plus_m
           ...    
33928    z_minus_m
8442     z_minus_m
1389     z_minus_m
24885    z_minus_m
37200    z_minus_m
Name: label, Length: 23360, dtype: object

In [35]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000020A64EADA60>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function tokenizer at 0x0000020A32B9B0D0>)),
                ('classifier', LinearSVC())])

In [43]:
samplePredict=pipe.predict(X_test)

In [45]:
pipe.predict(["Nie cierpię tej pracy chciałbym ją zmienić"])

array(['z_minus_m'], dtype=object)

In [47]:
print("Dokładność: ",pipe.score(X_test,Y_test))
#print("Dokładność: ",pipe.score(X_test,samplePredict))

Dokładność:  0.8102739726027397


In [38]:
print(classification_report(Y_test,samplePredict))

              precision    recall  f1-score   support

   z_minus_m       0.85      0.83      0.84      3430
    z_plus_m       0.76      0.79      0.77      2410

    accuracy                           0.81      5840
   macro avg       0.80      0.81      0.81      5840
weighted avg       0.81      0.81      0.81      5840

